In [53]:
%load_ext autoreload
%autoreload 2

import evolution
from config import Config
from pymoo.algorithms.moo.nsga2 import NSGA2
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 

config = Config()
problem = evolution.EVProblem(config)
algorithm = NSGA2(pop_size=config.pop_size,
                    sampling=evolution.SamplingAll(),
                    mutation=evolution.MutationAll(),
                    eliminate_duplicates=True)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
from pymoo.optimize import minimize

res = minimize(problem, 
                algorithm,
                callback=evolution.do_every_generations, 
                termination=config.termination)

2022-03-15 01:26:14.224085: I tensorflow/core/grappler/devices.cc:78] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-03-15 01:26:14.238005: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2022-03-15 01:26:14.246713: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:928] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.004ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.

2022-03-15 01:26:36.641387: I tensorflow/core/grappler/devices.cc:78] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-03-15 01:26:36.641512: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2022-03-15 01:26:36.649553: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:928]

generation = 1
population error: best = 0.06220000982284546, mean = 0.06400001049041748, median = 0.06470000743865967, worst = 0.06510001420974731


2022-03-15 01:27:22.487209: I tensorflow/core/grappler/devices.cc:78] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-03-15 01:27:22.487362: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2022-03-15 01:27:22.499611: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:928] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.004ms.
  function_optimizer: function_optimizer did nothing. time = 0.002ms.

2022-03-15 01:27:35.736121: I tensorflow/core/grappler/devices.cc:78] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-03-15 01:27:35.736365: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2022-03-15 01:27:35.748732: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:928]

generation = 2
population error: best = 0.05769997835159302, mean = 0.0614749938249588, median = 0.06174999475479126, worst = 0.06470000743865967


2022-03-15 01:28:23.463056: I tensorflow/core/grappler/devices.cc:78] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-03-15 01:28:23.465606: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2022-03-15 01:28:23.472709: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:928] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.004ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.

2022-03-15 01:28:40.212091: I tensorflow/core/grappler/devices.cc:78] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-03-15 01:28:40.212212: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2022-03-15 01:28:40.231560: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:928]

...

2022-03-15 01:28:54.613892: I tensorflow/core/grappler/devices.cc:78] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-03-15 01:28:54.614016: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2022-03-15 01:28:54.624823: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:928] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.003ms.
  function_optimizer: function_optimizer did nothing. time = 0.003ms.

2022-03-15 01:29:35.637945: I tensorflow/core/grappler/devices.cc:78] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-03-15 01:29:35.638083: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2022-03-15 01:29:35.652266: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:928]

generation = 4
population error: best = 0.05729997158050537, mean = 0.05954998731613159, median = 0.05934998393058777, worst = 0.06220000982284546


2022-03-15 01:30:09.114107: I tensorflow/core/grappler/devices.cc:78] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-03-15 01:30:09.114239: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2022-03-15 01:30:09.115995: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:928] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.005ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.



generation = 5
population error: best = 0.05260002613067627, mean = 0.05744999647140503, median = 0.057499974966049194, worst = 0.06220000982284546


2022-03-15 01:30:54.176944: I tensorflow/core/grappler/devices.cc:78] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-03-15 01:30:54.189411: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2022-03-15 01:30:54.195454: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:928] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.003ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.

